#Stream-Stream Joins using Structured Streaming (Python)
This notebook illustrates different ways of joining streams.

We are going to use the the canonical example of ad monetization, where we want to find out which ad impressions led to user clicks. 
Typically, in such scenarios, there are two streams of data from different sources - ad impressions and ad clicks. 
Both type of events have a common ad identifier (say, `adId`), and we want to match clicks with impressions based on the `adId`. 
In addition, each event also has a timestamp, which we will use to specify additional conditions in the query to limit the streaming state.


**Disclaimer** : Notebook นี้มาจาก [official docs ของ databricks](https://docs.databricks.com/aws/en/structured-streaming/examples#stream-stream-joins) ค่ะ อาจมีการปรับปรุงเปลี่ยนแปลงเล็กน้อยให้เข้ากับ free edition ในปัจจุบัน และอธิบายภาษาไทยตามความเข้าใจค่ะ
 
> Notebook นี้จะแสดงให้เราเห็นวิธี(ตั่งต่าง)ในการเชื่อม stream เข้าด้วยกัน
> 
> โดยใช้ตัวอย่างเป็นเรื่องของการสร้างรายได้จากการโฆษณา ซึ่งเราก็จะต้องหาว่า โฆษณาไหนที่ดึงดูดใจให้ลูกค้าคลิก
> 
> ในสถานการณ์แบบนี้ จะมีข้อมูลไหลมาจาก sources ที่ต่างกัน 2 sources คือ
> - ad impressions
> - ad clicks
> 
> และทั้ง events ทั้ง 2 อันนี้มีสิ่งระบุตัวของโฆษณาที่เหมือนกัน (`adId`) ดังนั้นเราก็ต้อง match clicks & impressions โดยใช้ `adId`
> 
> นอกจากนี้ ในแต่ละ event จะมี timestamp อยู่ด้วย ซึ่งจะใช้ระบุเงื่อนไขใน query เพื่อจะได้จำกัดตัว state ของการ stream

In absence of actual data streams, we are going to generate fake data streams using our built-in "rate stream", that generates data at a given fixed rate.

> ในกรณีนี้ที่เราไม่ได้มีข้อมูลจริง เราก็จะสร้างข้อมูลปลอมขึ้นมาโดยใช้ built-in 'rate stream' ซึ่งจะสร้างข้อมูลขึ้นมาใน rate ที่เรากำหนดไว้

In [0]:
from pyspark.sql.functions import rand

spark.conf.set("spark.sql.shuffle.partitions", "1")

impressions = (
  spark
    .readStream.format("rate").option("rowsPerSecond", "5").option("numPartitions", "1").load()
    .selectExpr("value AS adId", "timestamp AS impressionTime")
)

clicks = (
  spark
  .readStream.format("rate").option("rowsPerSecond", "10").option("numPartitions", "1").load()
  .where((rand() * 100).cast("integer") < 10)      # 10 out of every 100 impressions result in a click
  .selectExpr("(value-50) AS adId ", "timestamp AS clickTime")
  # -50 so that a click with same id as impression is generated later (i.e. delayed data).
  .where("adId > 0")
)    
  

Let's see what data these two streaming DataFrames generate.


> มาดูข้อมูลกันจ้ะ (แน่นอนว่า การจะ display ข้อมูลต้องมี checkpoint (อีกแล้ว) และไม่ใช่ checkpoint แบบซ้ำกัน เลยใช้การแก้ปัญหาเฉพาะหน้าเช่นเดิมด้วยการใช้ uuid)

In [0]:
import uuid
path = f"/Volumes/workspace/default/tutorial/testssetc/ststj_{uuid.uuid4()}"

display(impressions, checkpointLocation=path)

> ถ้าไม่เห็นข้อมูลส่วนนี้ออกมา ไม่ต้องตกใจ มาจาก logic ข้างบน (เพราะตอนแรกผู้ทดลองตกใจไปแล้วค่ะ555) 

logic คือ
- เราใช้เงื่อนไขตรง where ว่า `(rand() * 100).cast("integer") < 10` คือการสุ่มเลขมา และเลขที่สุ่มมาต้องน้อยกว่า 10 ด้วย ดังนั้นการสุ่มแบบนี้ก็เป็นเหมือนสุ่มกาชาหาเลขที่น้อยกว่า 10 เพื่อจะบอกว่าจะเก็บข้อมูลแต่ละแถวมั้ย ถ้าในแถวนั้นถูกสุ่มเลขมาแล้วเลขที่ได้มีค่าตั้งแต่ 10 ขึ้นไป = ทิ้งแถวนั้นไป (เหมือนที่เขาระบุใน cell ไว้ประมาณว่า จากทุก 100 impressions จะมีประมาณ 10 ที่นำไปสู่การ click ซึ่งก็สร้างสถานการณ์ให้มีความใกล้เคียงความเป็นจริง)
- การใช้ `value-50` เป็น `adId` มันจะสร้างข้อมูล 0-50, 1-50, 2-50,... ไปเรื่อยๆ นั่นแปลว่า มันจะครบ 50-50 ในจบวินาทีที่ 5 ถึงจะมีเริ่ม adId ออกมาในวินาทีที่ 6

จาก 2 ข้อนี้ทำให้ข้อมูลออกมาน้อยสะบัด และจาก path ที่ใช้บันทึก checkpoint รวมไปถึงข้อจำกัดต่างๆนานาของตัว free edition ที่ทำให้เราต้องใช้ volume มาเก็บข้อมูล เลยทำให้การรันโค้ด display impressions ใช้เวลาติ๊ดเดียว พอใช้เวลาติ๊ดเดียว ในส่วน clicks ก็ยังไม่ทันจะมีข้อมูลให้ใช้ เราเลยไม่เห็นข้อมูล T-T

ถ้าอยากเห็นข้อมูลสามารถแก้ logic ข้างบนได้!


In [0]:
path = f'/Volumes/workspace/default/tutorial/testssetc/ststj{uuid.uuid4()}'
display(clicks, checkpointLocation=path)

Note: 
- If you get an error saying the join is not supported, the problem may be that you are running this notebook in an older version of Spark. 
- If you are running on Community Edition, click Cancel above to stop the streams, as you do not have enough cores to run many streams simultaneously.

> Notes เพิ่มเติมจาก official
> - ถ้าได้ error ที่บอกว่าการ join ไม่ support มันแปลว่าเราใช้ spark ที่เก่าใน notebook นี้
> - ถ้าใช้บน community edition ให้ cencel เพื่อหยุด stream ด้วย เนื่องจาก core ของเราไม่พอที่จะใช้ stream หลายๆอันพร้อมกัน

### Inner Join

Let's join these two data streams. This is exactly the same as joining two batch DataFrames/Datasets by their common key `adId`.

> มาลองจอย 2 streams เข้าด้วยกันดู ซึ่งการ join ก็จะเหมือนกันกับการ join DF, Datasets (batch) โดยใช้คีย์ที่เหมือนกัน คือ `adId`

In [0]:
path = f'/Volumes/workspace/default/tutorial/testssetc/ststj{uuid.uuid4()}'

display(impressions.join(clicks, "adId"), checkpointLocation=path)

After you start this query, within a minute, you will start getting joined impressions and clicks. The delays of a minute is due to the fact that clicks are being generated with delay over the corresponding impressions.

In addition, if you expand the details of the query above, you will find a few timelines of query metrics - the processing rates, the micro-batch durations, and the size of the state. 
If you keep running this query, you will notice that the state will keep growing in an unbounded manner. This is because the query must buffer all past input as any new input can match with any input from the past. 

> หลังรัน query เพื่อ join ไปแล้วไม่นาน เราจะเริ่มเห็นข้อมูลที่มัน join กันของ impressions & clicks ซึ่งการที่เราต้องรอมันมาจาก delay ที่เราวางยาไว้

> ถ้าเราเปิดดูรายละเอียดของ query จะเห็น timeline พวก processing rate, ระยะเวลาที่ใช้ในแต่ละ micro-batch และ size ของ state

> การปล่อยให้ query มันรันไปเรื่อยๆ จะเห็นว่า state มันจะโตขึ้นเรื่อยๆ เพราะ query มันจะเก็บข้อมูลเก่าไว้ match กับข้อมูลใหม่ๆที่ไหลเข้ามา

(แต่ในเคสของเรา จะไม่ได้เห็นความเปลี่ยนแปลงตรงนี้)


### Inner Join with Watermarking

To avoid unbounded state, you have to define additional join conditions such that indefinitely old inputs cannot match with future inputs and therefore can be cleared from the state. In other words, you will have to do the following additional steps in the join.

1. Define watermark delays on both inputs such that the engine knows how delayed the input can be. 

1. Define a constraint on event-time across the two inputs such that the engine can figure out when old rows of one input is not going to be required (i.e. will not satisfy the time constraint) for matches with the other input. This constraint can be defined in one of the two ways.

  a. Time range join conditions (e.g. `...JOIN ON leftTime BETWEN rightTime AND rightTime + INTERVAL 1 HOUR`),
  
  b. Join on event-time windows (e.g. `...JOIN ON leftTimeWindow = rightTimeWindow`).

Let's apply these steps to our use case. 

1. Watermark delays: Say, the impressions and the corresponding clicks can be delayed/late in event-time by at most "10 seconds" and "20 seconds", respectively. This is specified in the query as watermarks delays using `withWatermark`.

1. Event-time range condition: Say, a click can occur within a time range of 0 seconds to 1 minute after the corresponding impression. This is specified in the query as a join condition between `impressionTime` and `clickTime`.



> เพื่อป้องกัน unbounded state เราจะต้องระบุเงื่อนไขในการ join เพิ่มเติม เพื่อไม่ให้ input เก่า(เกินไป)กะ input ใหม่(ในอนาคต) join กันได้แบบไม่จำกัดเวลา ละจะได้ clear จาก state หรือเอาง่ายๆคือเราสรุปว่ามี 2 ข้อเพิ่มเติมที่ต้องทำ คือ
1. ระบุลายน้ำ(?) Watermark การ delay ในทั้ง 2 input เพื่อให้ engine รู้ว่าข้อมูลที่เข้ามาจะ delay กันได้เท่าไหร่ยังไง
2. ระบุเงื่อนไขเวลาที่ engine มันจะตัดว่าแถวๆนี้ในข้อมูลชุดนึงไม่ได้จำเป็นที่จะเอามาใช้งานเพื่อ match กับข้อมูลอีกชุดแล้ว ซึ่งจะสามารถระบุได้ 2 แบบ
    - Time range join (ช่วงเวลา) : มองว่าเป็นเวลาเท่านี้ๆ คลาดกันได้ไม่เกินเท่านี้
    - กรอบช่วงเวลาตาม events (window) : มองเป็นกล่อง เป็นกรอบ ถ้าอยู่ในกรอบนี้คือ join คนละกรอบต่อให้เวลาห่างกันนิดเดียวก็ไม่ join

> Our Use Case
1. Watermark delays : impressions หน่วงได้ 10 วิ, clicks หน่วงได้ 20 วิ ระบุใน query โดยใช้ `withWatermark`
2. Event-time range : เราใช้เงื่อนไขเวลาแบบแรก ที่ใช้เป็นช่วงของเวลา คือ 1 คลิก สามารถเกิดขึ้นได้ใน range 0 วิ - 1 นาที **หลัง**จากที่ดูตัวโฆษณา ตรงนี้จะเป็นเงื่อนไขที่ใช้ระบุช่วงเวลาระหว่าง `impressionTime` and `clickTime`

In [0]:
from pyspark.sql.functions import expr

# Define watermarks
impressionsWithWatermark = impressions \
  .selectExpr("adId AS impressionAdId", "impressionTime") \
  .withWatermark("impressionTime", "10 seconds ")
clicksWithWatermark = clicks \
  .selectExpr("adId AS clickAdId", "clickTime") \
  .withWatermark("clickTime", "20 seconds")        # max 20 seconds late

path = f'/Volumes/workspace/default/tutorial/testssetc/ststj{uuid.uuid4()}'
# Inner join with time range conditions
display(
  impressionsWithWatermark.join(
    clicksWithWatermark,
    expr(""" 
      clickAdId = impressionAdId AND 
      clickTime >= impressionTime AND 
      clickTime <= impressionTime + interval 1 minutes    
      """
    )
  ), checkpointLocation=path
)

We are getting the similar results as the previous simple join query. However, if you look at the query metrics now, you will find that after about a couple of minutes of running the query, the size of the state will stabilize as the old buffered events will start getting cleared up.

> เราอาจจะเห็นว่ามันได้ผลลัพธ์แบบเดียวกันกับการ join แรกที่เราลองไป แต่ว่าถ้าไปดู detail เพิ่มเติมของ query เราจะเห็นว่าขนาดของ state ค่อนข้างจะคงที่ เพราะว่ามีการเคลียร์เหตุการณ์เก่าๆถูก clear

### Outer Joins with Watermarking 

Let's extend this use case to illustrate outer joins. Not all ad impressions will lead to clicks and you may want to keep track of impressions that did not produce clicks. This can be done by applying a left outer join on the impressions and clicks. The joined output will not have the matched clicks, but also the unmatched ones (with clicks data being NULL).

While the watermark + event-time constraints is optional for inner joins, for left and right outer joins they must be specified. This is because for generating the NULL results in outer join, the engine must know when an input row is not going to match with anything in future. Hence, the watermark + event-time constraints must be specified for generating correct results. 


> use case เพิ่มเติมของกรณีนี้ที่เหมาะที่ใช้ในการ join คือ outer join เพราะว่าไม่ใช่ทุก ads ที่จะมี click และเราก็อยากรู้อยากเห็นอยากจะ track ตัว ad พวกนี้ด้วยเช่นกัน เราจะใช้ left outer join ซึ่งจะมีทั้งการ click ที่ match กับ impressions และที่ไม่ match จะแสดงผล clicks เป็น null

> ในขณะที่ watermark และ event-time constraints เป็นเงื่อนไขที่จะมีหรือไม่มีก็ได้ใน inner joins แต่ว่า การระบุ 2 อย่างนี้ ต้องมีการระบุใน outer join เพราะว่าการที่ engine จะแสดงผล null ใน outer join ได้นั้น เจ้าตัว engine จะต้องรู้ว่า input แถวไหนที่เข้ามามันไม่ได้ match กับอะไรเลย เพื่อจะได้แสดงผลให้ถูกต้องนั่นเอง

In [0]:
from pyspark.sql.functions import expr

path = f'/Volumes/workspace/default/tutorial/testssetc/ststj{uuid.uuid4()}'
# Inner join with time range conditions
display(
  impressionsWithWatermark.join(
    clicksWithWatermark,
    expr(""" 
      clickAdId = impressionAdId AND 
      clickTime >= impressionTime AND 
      clickTime <= impressionTime + interval 1 minutes    
      """
    ),
    "leftOuter"
  ), checkpointLocation=path
)

After starting this query, you will start getting the inner results within a minute. But after a couple of minutes, you will also start getting the outer NULL results.

> หลังจากรัน query นี้ เราจะได้ผลให้ดูภายในเวลาสั้นๆ แต่หลังจากนั้นอีกประมาณ 2 นาที ถึงจะเริ่มเห็นผลที่มีค่า null ด้วย

### Further Information
You can read more about stream-stream joins in the following places:

- Databricks blog post on stream-stream joins - https://databricks.com/blog/2018/03/13/introducing-stream-stream-joins-in-apache-spark-2-3.html
- Apache Programming Guide on Structured Streaming - https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#stream-stream-joins
- Talk at Spark Summit Europe 2017 - https://databricks.com/session/deep-dive-into-stateful-stream-processing-in-structured-streaming
